# All imports necessary

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import sys

In [ ]:
sys.path.append('..')

In [ ]:
import os

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
from source.code.utils.utils import filter_by_subcorpus
from source.code.utils.utils import get_tagged_texts_as_pd

from source.code.transformers.sentenceextractor import SentenceExtractor
from source.code.models.memorytagger import MemoryTagger
from source.code.models.bilstmtagger import BiLSTMTagger

from source.code.utils.preprocessing import filtrations
from source.code.utils.preprocessing import additional_features
from source.code.utils.preprocessing import crf_pre_processing

In [ ]:
from sklearn_crfsuite.metrics import flat_classification_report as crfsuite_classification_report

from sklearn_crfsuite import metrics

from sklearn_crfsuite import CRF as sklearn_crf

In [ ]:
from seqeval.metrics import classification_report as seqeval_classification_report

In [ ]:
from sklearn.metrics import classification_report

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
features = ['semantic_relation_tagged', 'animacy_tagged', 'lambda_dsr_len', 'word_sense_exists', 'is_title', 'contains_digits', 'word_len']
target = 'ner_tag'

In [ ]:
pd.set_option('display.max_rows', 30000)

# Naive tag frequency memorization

In [ ]:
target_subcorpus_folders = filter_by_subcorpus('../data/datasets/gmb-2.2.0/', 'subcorpus: Voice of America')

In [ ]:
X = get_tagged_texts_as_pd(target_subcorpus_folders, '../data/datasets/gmb-2.2.0/')

In [ ]:
X.ner_tag.value_counts()

In [ ]:
X = filtrations(X)

In [ ]:
X.ner_tag.value_counts()

In [ ]:
X = additional_features(X)

In [ ]:
X.info()

In [ ]:
X, y = X.token, X.ner_tag

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

In [ ]:
estimator = MemoryTagger()

In [ ]:
scores = cross_val_score(estimator, X_train, y_train, cv=5, verbose=True, n_jobs=-1)

In [ ]:
scores

In [ ]:
estimator.fit(X_train, y_train)

In [ ]:
report = classification_report(y_pred=estimator.predict(X_test), y_true=y_test)
print(report)

# Random Forest Classifier

In [ ]:
target_subcorpus_folders = filter_by_subcorpus('../data/datasets/gmb-2.2.0/', 'subcorpus: Voice of America')

In [ ]:
X = get_tagged_texts_as_pd(target_subcorpus_folders, '../data/datasets/gmb-2.2.0/')

In [ ]:
X.ner_tag.value_counts()

In [ ]:
X = filtrations(X)

In [ ]:
X.ner_tag.value_counts()

In [ ]:
X = additional_features(X)

In [ ]:
X.info()

In [ ]:
X, y = X[features], X[target]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

In [ ]:
estimator = RandomForestClassifier()

In [ ]:
scores = cross_val_score(estimator, X_train, y_train, cv=5, verbose=True, n_jobs=-1)

In [ ]:
scores

In [ ]:
estimator.fit(X_train, y_train)

In [ ]:
report = classification_report(y_pred=estimator.predict(X_test), y_true=y_test)
print(report)

# HMM

TBD...

# CRF

In [ ]:
target_subcorpus_folders = filter_by_subcorpus('../data/datasets/gmb-2.2.0/', 'subcorpus: Voice of America')

In [ ]:
X = get_tagged_texts_as_pd(target_subcorpus_folders, '../data/datasets/gmb-2.2.0/')

In [ ]:
X.ner_tag.value_counts()

In [ ]:
X = filtrations(X, with_dots=True)

In [ ]:
X.ner_tag.value_counts()

In [ ]:
X = additional_features(X)

In [ ]:
X.info()

In [ ]:
X, y = crf_filtration_and_pre_processing(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
estimator = sklearn_crf(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100, all_possible_transitions=False)

In [ ]:
scores = cross_val_score(estimator, X_train, y_train, cv=5, verbose=True, n_jobs=-1)

In [ ]:
scores

In [ ]:
estimator.fit(X_train, y_train)

In [ ]:
report = flat_classification_report(y_pred=estimator.predict(X_test), y_true=y_test)
print(report)

# Bi-LSTM

In [ ]:
target_subcorpus_folders = filter_by_subcorpus('../data/datasets/gmb-2.2.0/', 'subcorpus: Voice of America')

In [ ]:
X = get_tagged_texts_as_pd(target_subcorpus_folders, '../data/datasets/gmb-2.2.0/')

In [ ]:
X.head(30)

In [ ]:
X.ner_tag.value_counts()

In [ ]:
X = filtrations(X, with_dots=True)

In [ ]:
X.ner_tag.value_counts()

In [ ]:
X.ner_tag.nunique()

In [ ]:
X, y = SentenceExtractor().fit_transform(X)

In [ ]:
n_tags = 17

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
estimator = BiLSTMTagger(checkpoint_dir='../data/datasets/keras_model/')

In [ ]:
estimator.fit(X_train, y_train)

In [ ]:
y_pred = estimator.predict(X_test)

In [ ]:
y_test = [[estimator.tag2idx[w] for w in s] for s in y_test]
y_test = pad_sequences(maxlen=75, sequences=y_test, padding="post", value=estimator.tag2idx["O"])
y_test = [[estimator.idx2tag[w] for w in s] for s in y_test]

In [ ]:
crfsuite_report = crfsuite_classification_report(y_pred=y_pred, y_true=y_test)
print(crfsuite_report)

In [ ]:
seqeval_report = seqeval_classification_report(y_pred=y_pred, y_true=y_test)
print(seqeval_report)

# Conclusion